In [ ]:
# Importação dos pacotes python
import pandas as pd
import matplotlib.pyplot as mlp
import numpy as np

In [ ]:
# Carregando o arquivo
arquivo = "dados_compras.json"
base = pd.read_json(arquivo, orient = "records")
base.head()# retorna as 5 primeiras linhas da base de dados

## Informações Sobre os Consumidores

In [ ]:
#localizar se nos dados há mais de uma venda por consumidor
dados_demo = base.loc[:,["Login","Sexo","Idade"]] #retornar todas as linhas das colunas selecionadas
dados_demo = dados_demo.drop_duplicates()#retira todas os usuário que tiveram mais de uma compra
dados_demo.head()

In [ ]:
total_usuários = dados_demo["Login"].count()
quantidade_vendas = base["Valor"].count()
media_vendas = base["Valor"].mean()# graneza escalar por ter uma unidade de medida
total_vendas = base["Valor"].sum()

#criar um dataframe para juntar todas as variáveis criadas

vendas = pd.DataFrame({"Quantidade total de usuários":total_usuários,
                      "Quantidade de Vendas":quantidade_vendas,
                      "Valor Médio de Vendas": [media_vendas],#valor escalar entre []
                      "Total de vendas":total_vendas
                       })

#ajustar formato dos numeros com função map
vendas["Quantidade total de usuários"] = vendas["Quantidade total de usuários"].map("R$ {:,.2f}".format)
vendas["Quantidade de Vendas"] = vendas["Quantidade de Vendas"].map("{:}".format)
vendas["Valor Médio de Vendas"] = vendas["Valor Médio de Vendas"].map("R$ {:,.2f}".format)
vendas["Total de vendas"] = vendas["Total de vendas"].map("R$ {:,.2f}".format)

display(vendas)#display apresenta os dados em formato de tabela
#todo - Alinhar texto do dataframe


## Análise Geral de Compras

In [ ]:
# Implemente aqui sua solução

## Análise Demográfica

In [ ]:
# Implemente aqui sua solução

## Informações Demográficas Por Gênero

In [ ]:
# Implemente aqui sua solução

## Análise de Compras Por Gênero

In [ ]:
# Implemente aqui sua solução

## Consumidores Mais Populares (Top 5)

In [ ]:
# Implemente aqui sua solução

## Itens Mais Populares

In [ ]:
# Implemente aqui sua solução

## Itens Mais Lucrativos

In [ ]:
# Implemente aqui sua solução